In [ ]:
!curl --request GET --url host.docker.internal:21434/api/tags

In [ ]:
from langchain_ollama import ChatOllama

### llm 
local_llm="deepseek-r1:1.5b-qwen-distill-fp16"
llm = ChatOllama(model=local_llm, temperature=0, base_url="http://host.docker.internal:21434")
llm_json = ChatOllama(model=local_llm, temperature=0, return_json=True, base_url="http://host.docker.internal:21434")

In [ ]:
msg = llm.invoke("What is the capital of France?")
msg.content

__________________

# GROQ test

In [ ]:
import os

from groq import Groq
from dotenv import load_dotenv
load_dotenv("../../groq.env")


In [ ]:
client = Groq(
    # This is the default and can be omitted
    api_key=os.getenv("GROQ_API_KEY"),
)

In [ ]:
model_name = os.getenv("GROQ_LLM")
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "you are a helpful assistant."
        },
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model=model_name,
    reasoning_format={"type": "groq"},
    temperature=0.6,
    max_completion_tokens=131072,
    top_p=0.95,
    stream=True,
    stop=None,
    
)

In [ ]:

for chunk in chat_completion:
    print(chunk.choices[0].delta.content or "", end="")

__________


# GROQ langchain test

In [ ]:
import os
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from ollama_deep_researcher.prompts import (
    query_writer_instructions,
    summarizer_instructions,
    reflection_instructions,
    get_current_date,
)
from dotenv import load_dotenv
load_dotenv("../../groq.env")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
model_name = os.getenv("GROQ_LLM")

In [ ]:
current_date = get_current_date()
formatted_prompt = query_writer_instructions.format(
        current_date=current_date, research_topic="fast language models"
    )

In [ ]:
llm = ChatGroq(
    model=model_name,
    temperature=0,
    response_format={"type": "json_object"},
)
#llm_json_mode = llm.with_structured_output(method="json_mode")

In [ ]:
result = llm_json_mode.invoke(
    [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content=f"Generate a query for web search:"),
    ]
)

In [ ]:
result2 = llm.invoke(
    [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content=f"Generate a query for web search:"),
    ]
)

In [ ]:
print(result2.content)

_____

# tavily test (Used for making the crawler search)

In [ ]:
import os
from ollama_deep_researcher.utils import (
    deduplicate_and_format_sources,
    tavily_search,
    format_sources,
)
from dotenv import load_dotenv
load_dotenv("../../.env")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [ ]:
search_results = tavily_search("what is the capital of France?", fetch_full_page=True, max_results=3)

In [ ]:
search_results

In [ ]:
search_str = deduplicate_and_format_sources(
            search_results,
            max_tokens_per_source=10000000,
            fetch_full_page=True,
        )

In [ ]:
print(search_str)

_______

# Test rets_info_scraper

In [1]:
from ollama_deep_researcher.retsinfo_crawl import retsinfo_search_and_crawl, retsinfo_search, extract_ids

In [ ]:
if __name__ == "__main__":
    search_r = await retsinfo_search("Justitsministerens beføjelser til fastsættelse af regler for indsamlinger", max_results=3)

In [ ]:
search_r

In [ ]:
if __name__ == "__main__":
    id_url_pari = await extract_ids(search_r)

In [ ]:
id_url_pari.values()

In [3]:
if __name__ == "__main__":
    # Load environment variables from .env file
   result = await retsinfo_search_and_crawl(query="Justitsministerens beføjelser til fastsættelse af regler for indsamlinger", max_results=3, crawler_base_url="http://localhost:11235")

    

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://www.retsinformation.dk/eli/lta/2006/542                                                      |
✓ | ⏱: 3.35s 

[SCRAPE].. ◆ https://www.retsinformation.dk/eli/lta/2006/542                                                      |
✓ | ⏱: 0.06s 

[COMPLETE] ● https://www.retsinformation.dk/eli/lta/2006/542                                                      |
✓ | ⏱: 3.41s 

[FETCH]... ↓ https://www.retsinformation.dk/eli/lta/2020/160                                                      |
✓ | ⏱: 4.76s 

...

[COMPLETE] ● https://www.retsinformation.dk/eli/retsinfo/2016/10488                                               |
✓ | ⏱: 7.51s 

[FETCH]... ↓ https://www.retsinformation.dk/eli/retsinfo/2017/9228                                                |
✓ | ⏱: 7.99s 

[SCRAPE].. ◆ https://www.retsinformation.dk/eli/retsinfo/2017/9228                                                |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.retsinformation.dk/eli/retsinfo/2017/9228                                                |
✓ | ⏱: 8.01s 

https://www.retsinformation.dk/eli/lta/2006/542 crawled OK!
https://www.retsinformation.dk/eli/lta/2020/160 crawled OK!
https://www.retsinformation.dk/eli/lta/2018/1706 crawled OK!
https://www.retsinformation.dk/eli/lta/2017/671 crawled OK!
https://www.retsinformation.dk/eli/lta/2025/431 crawled OK!
https://www.retsinformation.dk/eli/lta/2014/511 crawled OK!
https://www.retsinformation.dk/eli/lta/2017/1287 crawled OK!
https://www.retsinformation.dk/eli/lta/2025/285 crawled OK!
https://www.retsinformation.dk/eli/lta/2020/1035 crawled OK!
https://www.retsinformation.dk/eli/lta/2023/743 crawled OK!
https://www.retsinformation.dk/eli/lta/2021/1321 crawled OK!
https://www.retsinformation.dk/eli/lta/2021/2166 crawled OK!
https://www.retsinformation.dk/eli/lta/2017/231 crawled OK!
https://www.retsinformation.dk/eli/lta/2019/81 crawled OK!
https://www.retsinformation.dk/eli/lta/2022/29 crawled OK!
https://www.retsinformation.dk/eli/lta/2024/118 crawled OK!
https://www.retsinformation.dk/eli/lt